In [1]:
#install Kaggle Command Line Tool to use on Jupyter Notebook -here-
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [2]:
#verify it works
!kaggle --version 

Kaggle API 1.6.14


In [3]:
#check commands you can use
!kaggle --help

usage: kaggle [-h] [-v]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set, unset}
    competitions (c)    Commands related to Kaggle 

In [4]:
#view datasets comman
!kaggle datasets -h

usage: kaggle datasets [-h]
                       {list,files,download,create,version,init,metadata,status}
                       ...

options:
  -h, --help            show this help message and exit

commands:
  {list,files,download,create,version,init,metadata,status}
    list                List available datasets
    files               List dataset files
    download            Download dataset files
    create              Create a new dataset
    version             Create a new dataset version
    init                Initialize metadata file for dataset creation
    metadata            Download metadata about a dataset
    status              Get the creation status for a dataset


In [5]:
#list available commands to search for dataset
!kaggle datasets list -h


usage: kaggle datasets list [-h] [--sort-by SORT_BY] [--size SIZE]
                            [--file-type FILE_TYPE] [--license LICENSE_NAME]
                            [--tags TAG_IDS] [-s SEARCH] [-m] [--user USER]
                            [-p PAGE] [-v] [--max-size MAX_SIZE]
                            [--min-size MIN_SIZE]

options:
  -h, --help            show this help message and exit
  --sort-by SORT_BY     Sort list results. Default is 'hottest'. Valid options are 'hottest', 'votes', 'updated', and 'active'
  --size SIZE           DEPRECATED. Please use --max-size and --min-size to filter dataset sizes.
  --file-type FILE_TYPE
                        Search for datasets with a specific file type. Default is 'all'. Valid options are 'all', 'csv', 'sqlite', 'json', and 'bigQuery'. Please note that bigQuery datasets cannot be downloaded
  --license LICENSE_NAME
                        Search for datasets with a specific license. Default is 'all'. Valid options are 'all', 'c

In [6]:
# Search datasets
!kaggle datasets list

401 - Unauthorized - Unauthenticated


In [7]:
# Search datasets for aqi or air quality
!kaggle datasets list -s aqi

401 - Unauthorized - Unauthenticated


In [8]:
#use !kaggle datasets files <owner>/<dataset-name> 

!kaggle datasets files azminetoushikwasi/aqi-air-quality-index-scheduled-daily-update

name            size  creationDate         
-------------  -----  -------------------  
data_date.csv  418KB  2023-08-10 13:58:38  


In [9]:
#look at download options 
!kaggle datasets download -h

usage: kaggle datasets download [-h] [-f FILE_NAME] [-p PATH] [-w] [--unzip]
                                [-o] [-q]
                                [dataset]

options:
  -h, --help            show this help message and exit
  dataset               Dataset URL suffix in format <owner>/<dataset-name> (use "kaggle datasets list" to show options)
  -f FILE_NAME, --file FILE_NAME
                        File name, all files downloaded if not provided
                        (use "kaggle datasets files -d <dataset>" to show options)
  -p PATH, --path PATH  Folder where file(s) will be downloaded, defaults to current working directory
  -w, --wp              Download files to current working path
  --unzip               Unzip the downloaded file. Will delete the zip file when completed.
  -o, --force           Skip check whether local version of file is up to date, force file download
  -q, --quiet           Suppress printing information about the upload/download progress


In [10]:
#download specific file
!kaggle datasets download azminetoushikwasi/aqi-air-quality-index-scheduled-daily-update -f data_date.csv 

Dataset URL: https://www.kaggle.com/datasets/azminetoushikwasi/aqi-air-quality-index-scheduled-daily-update
License(s): ODbL-1.0
100%|█████████████████████████████████████████| 418k/418k [00:00<00:00, 850kB/s]
100%|█████████████████████████████████████████| 418k/418k [00:00<00:00, 847kB/s]


In [11]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
aqi = pd.read_csv("data_date.csv")
aqi


,Date,Country,Status,AQI Value
0,2022-07-21,Albania,Good,14
1,2022-07-21,Algeria,Moderate,65
2,2022-07-21,Andorra,Moderate,55
3,2022-07-21,Angola,Unhealthy for Sensitive Groups,113
4,2022-07-21,Argentina,Moderate,63
...,...,...,...,...
12981,2023-08-10,Uzbekistan,Good,28
12982,2023-08-10,Vatican,Good,34
12983,2023-08-10,Venezuela,Good,26
12984,2023-08-10,Vietnam,Unhealthy for Sensitive Groups,142


In [13]:
#install sql
import sqlite3

In [14]:
#create database called "aqi.db"
db_connect = sqlite3.connect('aqi.db')

In [15]:
#cur (cursors) allow us the execute sql queries! 
cur = db_connect.cursor()

In [16]:
#create table "aqitable" with composite unique constraint
cur.execute("""
    CREATE TABLE IF NOT EXISTS aqitable (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        Date DATETIME NOT NULL,
        Country TEXT,
        Status TEXT,
        AQI_Value INTEGER,
        UNIQUE(Date, Country)
    )
""")


In [17]:
#show all new tables in new SQL database 
cur.execute ("SELECT name from sqlite_master where type='table' order by name;")
results = cur.fetchall()
print(results) 

[('aqitable',), ('sqlite_sequence',)]


In [18]:
#pandas allow to run SQL querry and save data 

import pandas as pd

In [19]:
#Put AQI dataset we found into our sql data as aqi 
aqi.to_sql('aqitable', db_connect, if_exists='replace', index = False)

12986

In [20]:
#query sql tabke inty pythin pandas dataframe
latest_aqi = pd.read_sql_query('select * from aqitable limit 100', db_connect)

In [21]:
#view latest aqi data
latest_aqi

,Date,Country,Status,AQI Value
0,2022-07-21,Albania,Good,14
1,2022-07-21,Algeria,Moderate,65
2,2022-07-21,Andorra,Moderate,55
3,2022-07-21,Angola,Unhealthy for Sensitive Groups,113
4,2022-07-21,Argentina,Moderate,63
...,...,...,...,...
95,2022-07-21,Netherlands,Moderate,70
96,2022-07-21,New Caledonia,Good,15
97,2022-07-21,New Zealand,Good,18
98,2022-07-21,Nigeria,Unhealthy for Sensitive Groups,105


In [22]:
import pandas as pd

# Define the correct paths for the CSV files
aqi_file_path = '/Users/bellewills/data/data_date.csv'
weather_file_path = '/Users/bellewills/data/GlobalWeatherRepository.csv'
pollution_file_path = '/Users/bellewills/data/global air pollution dataset.csv'

# Load datasets into pandas DataFrames
df_aqi = pd.read_csv(aqi_file_path)
df_weather = pd.read_csv(weather_file_path)
df_pollution = pd.read_csv(pollution_file_path)

# Print column names to identify correct names
print("AQI DataFrame columns:", df_aqi.columns)
print("Weather DataFrame columns:", df_weather.columns)
print("Pollution DataFrame columns:", df_pollution.columns)


AQI DataFrame columns: Index(['Date', 'Country', 'Status', 'AQI Value'], dtype='object')
Weather DataFrame columns: Index(['country', 'location_name', 'latitude', 'longitude', 'timezone',
       'last_updated_epoch', 'last_updated', 'temperature_celsius',
       'temperature_fahrenheit', 'condition_text', 'wind_mph', 'wind_kph',
       'wind_degree', 'wind_direction', 'pressure_mb', 'pressure_in',
       'precip_mm', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius',
       'feels_like_fahrenheit', 'visibility_km', 'visibility_miles',
       'uv_index', 'gust_mph', 'gust_kph', 'air_quality_Carbon_Monoxide',
       'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
       'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10',
       'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'sunrise',
       'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination'],
      dtype='object')
Pollution DataFrame columns: Index(['Country', 'City', 'AQI Value',

In [23]:
import pandas as pd
import sqlite3

# Define the correct paths for the CSV files
aqi_file_path = '/Users/bellewills/data/data_date.csv'
weather_file_path = '/Users/bellewills/data/GlobalWeatherRepository.csv'
pollution_file_path = '/Users/bellewills/data/global air pollution dataset.csv'

# Load datasets into pandas DataFrames
df_aqi = pd.read_csv(aqi_file_path)
df_weather = pd.read_csv(weather_file_path)
df_pollution = pd.read_csv(pollution_file_path)

# Print column names to identify correct names
print("AQI DataFrame columns:", df_aqi.columns)
print("Weather DataFrame columns:", df_weather.columns)
print("Pollution DataFrame columns:", df_pollution.columns)

# Create a SQLite database
conn = sqlite3.connect('aqi_database.db')

# Create and populate the locations table
df_locations_aqi = df_aqi[['Country']].drop_duplicates().reset_index(drop=True)
df_locations_weather = df_weather[['country', 'location_name', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
df_locations_pollution = df_pollution[['Country', 'City']].drop_duplicates().reset_index(drop=True)

df_locations_aqi.rename(columns={'Country': 'country'}, inplace=True)
df_locations_weather.rename(columns={'country': 'country', 'location_name': 'city'}, inplace=True)
df_locations_pollution.rename(columns={'Country': 'country', 'City': 'city'}, inplace=True)

# Combine all locations into one DataFrame
df_locations = pd.concat([df_locations_aqi, df_locations_weather, df_locations_pollution]).drop_duplicates().reset_index(drop=True)
df_locations['location_id'] = df_locations.index + 1

# Create and populate the measurements table
df_measurements_aqi = df_aqi[['Country', 'Date', 'AQI Value']]
df_measurements_aqi = df_measurements_aqi.rename(columns={'Country': 'country', 'Date': 'date', 'AQI Value': 'value'})
df_measurements_aqi['parameter'] = 'AQI'
df_measurements_aqi['source'] = 'aqi'

df_measurements_weather = df_weather[['country', 'location_name', 'temperature_celsius', 'last_updated']]
df_measurements_weather = df_measurements_weather.rename(columns={'country': 'country', 'location_name': 'city', 'temperature_celsius': 'value', 'last_updated': 'date'})
df_measurements_weather['parameter'] = 'Temperature'
df_measurements_weather['source'] = 'weather'

df_measurements_pollution = df_pollution[['Country', 'City', 'CO AQI Value', 'NO2 AQI Value', 'Ozone AQI Value', 'PM2.5 AQI Value']]
df_measurements_pollution = df_measurements_pollution.melt(id_vars=['Country', 'City'], var_name='parameter', value_name='value')
df_measurements_pollution = df_measurements_pollution.rename(columns={'Country': 'country', 'City': 'city'})
df_measurements_pollution['source'] = 'pollution'

df_measurements = pd.concat([df_measurements_aqi, df_measurements_weather, df_measurements_pollution]).drop_duplicates().reset_index(drop=True)
df_measurements = df_measurements.merge(df_locations[['location_id', 'country', 'city']], on=['country', 'city'])

# Create and populate the parameters table
df_parameters = df_measurements[['parameter']].drop_duplicates().reset_index(drop=True)
df_parameters['parameter_id'] = df_parameters.index + 1

# Merge parameter_id into measurements
df_measurements = df_measurements.merge(df_parameters, on='parameter')

# Save DataFrames to SQL tables
df_locations.to_sql('locations', conn, if_exists='replace', index=False)
df_measurements.to_sql('measurements', conn, if_exists='replace', index=False)
df_parameters.to_sql('parameters', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()


AQI DataFrame columns: Index(['Date', 'Country', 'Status', 'AQI Value'], dtype='object')
Weather DataFrame columns: Index(['country', 'location_name', 'latitude', 'longitude', 'timezone',
       'last_updated_epoch', 'last_updated', 'temperature_celsius',
       'temperature_fahrenheit', 'condition_text', 'wind_mph', 'wind_kph',
       'wind_degree', 'wind_direction', 'pressure_mb', 'pressure_in',
       'precip_mm', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius',
       'feels_like_fahrenheit', 'visibility_km', 'visibility_miles',
       'uv_index', 'gust_mph', 'gust_kph', 'air_quality_Carbon_Monoxide',
       'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
       'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10',
       'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'sunrise',
       'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination'],
      dtype='object')
Pollution DataFrame columns: Index(['Country', 'City', 'AQI Value',

In [24]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('aqi_database.db')

# Function to list all tables
def list_tables(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    result = conn.execute(query).fetchall()
    tables = [row[0] for row in result]
    return tables

# Function to get a preview of a table
def preview_table(conn, table_name, limit=5):
    query = f"SELECT * FROM {table_name} LIMIT {limit};"
    df = pd.read_sql_query(query, conn)
    return df

# List all tables in the database
tables = list_tables(conn)
print("Tables in the database:", tables)

# Preview each table
for table in tables:
    print(f"\nPreview of {table} table:")
    print(preview_table(conn, table))

# Close the connection
conn.close()


Tables in the database: ['sqlite_sequence', 'locations', 'measurements', 'parameters']

Preview of sqlite_sequence table:
Empty DataFrame
Columns: [name, seq]
Index: []

Preview of locations table:
     country  city latitude longitude  location_id
0    Albania  None     None      None            1
1    Algeria  None     None      None            2
2    Andorra  None     None      None            3
3     Angola  None     None      None            4
4  Argentina  None     None      None            5

Preview of measurements table:
     country        date  value parameter source  city  location_id  \
0    Albania  2022-07-21   14.0       AQI    aqi  None            1   
1    Algeria  2022-07-21   65.0       AQI    aqi  None            2   
2    Andorra  2022-07-21   55.0       AQI    aqi  None            3   
3     Angola  2022-07-21  113.0       AQI    aqi  None            4   
4  Argentina  2022-07-21   63.0       AQI    aqi  None            5   

   parameter_id  
0             1  
1

In [25]:
%pip install flask

Note: you may need to restart the kernel to use updated packages.


In [26]:
!python3.11 -m pip install --upgrade pip

In [1]:
from flask import Flask, render_template
import pandas as pd
import sqlite3

app = Flask(__name__, template_folder='templates')

def get_db_connection():
    conn = sqlite3.connect('aqi_database.db')
    conn.row_factory = sqlite3.Row
    return conn

def list_tables(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]
    return tables

@app.route('/')
def index():
    conn = get_db_connection()
    tables = list_tables(conn)
    table_data = {}
    for table in tables:
        query = f"SELECT * FROM {table};"
        df = pd.read_sql_query(query, conn)
        table_data[table] = df.to_html(classes='data', header=True, index=False)
    conn.close()
    return render_template('index.html', tables=table_data)

if __name__ == "__main__":
    app.run(debug=True, port=5100)  # Change the port number to an unused port


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5100
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/bellewills/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/bellewills/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/bellewills/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/bellewills/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/Users/bellewills/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
   

SystemExit: 1

/Users/bellewills/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
